In [1]:
import smf2021.rank2 as rank2
import smf2021.Optimiser as O
from smf.db_engine import DbEngine
import eikon as ek
# from smf.ranker import retrieve
# from smf.ranker import rank
# from smf.ranker import add_sector_info
from smf.db_engine import DbEngine
ek.set_app_key('403255e90c7647afafbfb5c0000d60ac4c8cc536')

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf

from numpy import log
from dateutil.parser import parse
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import pmdarima as pm
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns
from pandas.api.types import is_numeric_dtype
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import time
import datetime as dt


In [2]:
# Industrials
ind_fields = ['TR.GICSSector','TR.PE',"TR.PriceToBVPerShare",
              "TR.EPSMean", "TR.EVToSales", "TR.EVToEBIT",
              "TR.PtoEBTMeanEst", "TR.FwdPEG"]


# Comms
comms_fields = ['TR.GICSSector','TR.ROATotalAssetsPercent',
                'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                'TR.EVToEBITDA',
                'TR.ROICActValue', 'TR.TimesInterestEarned', 'TR.FCFMeanYield',
               ]

# Healthcare
health_fields = ['TR.GICSSector','TR.PE',  'TR.EVToEBITDA',
                 'TR.EVToSales', 'TR.ROICActValue', 'TR.FCFMeanYield',
'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                 "TR.PriceToBVPerShare"]

cons_fields = ['TR.GICSSector','TR.PE', 'TR.ROATotalAssetsPercent', 
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.EVToEBITDA', 'TR.EVToSales']

fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", "TR.EPSMean",
          "TR.EVToSales", "TR.EVToEBIT",
          "TR.PtoEBTMeanEst", "TR.FwdPEG"]  

tech_fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", 
               'TR.ROICActValue', 'TR.ReinvestmentRatePct',
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.FCFMeanYield']

In [3]:
ind_ranker = rank2.ranker(ind_fields)
comms_ranker = rank2.ranker(comms_fields)
health_ranker = rank2.ranker(health_fields)
cons_ranker = rank2.ranker(cons_fields)
tech_ranker = rank2.ranker(tech_fields)
all_comms = comms_ranker.get_data()
all_ind = ind_ranker.get_data()
all_health = health_ranker.get_data()
all_cons = cons_ranker.get_data()
all_tech = tech_ranker.get_data()
all_tech = tech_ranker.get_data()

In [6]:
# True => lower is better.
# ROIC < WACC => remove from list.
# Interest Coverage Ratio > 1.5
ind_data = ind_ranker.industry_data('Industrials')
comms_data = comms_ranker.industry_data('Communication Services')
cons_data = cons_ranker.industry_data('Consumer Staples')
health_data = health_ranker.industry_data('Health Care')
tech_data = health_ranker.industry_data('Information Technology')
asc_ind = {'P/E (Daily Time Series Ratio)': True,
           'Price To Book Value Per Share (Daily Time Series Ratio)': True,
           'Earnings Per Share - Mean': False, 
           'Enterprise Value To Sales (Daily Time Series Ratio)': True,
           'Enterprise Value To EBIT (Daily Time Series Ratio)': False, 
           'Price / EBITDA (Mean Estimate)': True, 
           'Forward P/E/G (Daily Time Series Ratio)': True}
             
asc_comms = {'ROA Total Assets, Percent': False,
             'ROE Common Equity %': False,
             'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
             'Return On Invested Capital - Actual': False,
             'Interest Coverage Ratio': False,
             'Free Cash Flow Yield %': False}


asc_tech = {'P/E (Daily Time Series Ratio)': True,
            'ROE Common Equity %': False,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Enterprise Value To Sales (Daily Time Series Ratio)': True,
            'Price To Book Value Per Share (Daily Time Series Ratio)': True,

            'Return On Invested Capital - Actual': False,
            'Free Cash Flow Yield %': False}

asc_health = {'P/E (Daily Time Series Ratio)': True,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Enterprise Value To Sales (Daily Time Series Ratio)': True,
            'Return On Invested Capital - Actual': False,
            'Free Cash Flow Yield %': False,
            'ROE Common Equity %': False,
            'Price To Book Value Per Share (Daily Time Series Ratio)': True}

asc_cons = {'P/E (Daily Time Series Ratio)': True,
                'ROA Total Assets, Percent': False,
                'ROE Common Equity %': False,
                'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
                'Enterprise Value To Sales (Daily Time Series Ratio)': True
                }

In [7]:
ind_z_scores = ind_ranker.z_scores(ind_data, asc_ind)
health_z_scores = health_ranker.z_scores(health_data, asc_health)
comms_z_scores = comms_ranker.z_scores(comms_data, asc_comms)
tech_z_scores = tech_ranker.z_scores(tech_data, asc_tech)
cons_z_scores = cons_ranker.z_scores(cons_data, asc_cons)
cons_list = cons_z_scores.head(5)['Instrument'].tolist()
ind_list = ind_z_scores.head(5)['Instrument'].tolist()
health_list = health_z_scores.head(5)['Instrument'].tolist()
comms_list = comms_z_scores.head(5)["Instrument"].tolist()
tech_list = tech_z_scores.head(5)["Instrument"].tolist()

stocks_list = cons_list+comms_list+ind_list+health_list+tech_list
stocks_list


['HFGG.DE',
 'HNKG_p.DE',
 'BEIG.DE',
 'DTEGn.DE',
 'DPWGn.DE',
 'ENR1n.DE',
 'BNRGn.DE',
 'SIEGn.DE',
 'AIRG.DE',
 'MRCG.DE',
 'FREG.DE',
 'FMEG.DE',
 'SHLG.DE',
 'QIA.DE',
 'SAPG.DE',
 'IFXGn.DE']

Optimiser 1

In [8]:
final_companies1 = stocks_list
data=ek.get_timeseries(final_companies1, fields='CLOSE', start_date='2021-01-19', end_date='2022-01-19' )
close_prices_12m = data.fillna(method="ffill")
close_prices_12m =close_prices_12m ._get_numeric_data()
close_prices_12m=close_prices_12m.astype(float)

In [9]:
close_prices_12m

CLOSE,HFGG.DE,HNKG_p.DE,BEIG.DE,DTEGn.DE,DPWGn.DE,ENR1n.DE,BNRGn.DE,SIEGn.DE,AIRG.DE,MRCG.DE,FREG.DE,FMEG.DE,SHLG.DE,QIA.DE,SAPG.DE,IFXGn.DE
Date,,,,,,,,,,,,,,,,
2021-04-29,70.54,96.46,93.68,15.950,48.590,28.04,74.94,139.40,99.97,145.55,39.915,65.26,47.30,40.11,118.04,33.680
2021-04-30,69.00,95.56,93.90,16.000,48.955,27.80,74.68,138.82,99.88,146.15,40.880,66.16,47.48,40.53,116.76,33.565
2021-05-03,70.06,95.90,93.92,16.098,49.700,28.03,75.10,140.84,99.85,146.45,41.085,66.84,48.42,39.99,116.62,33.720
2021-05-04,65.38,94.70,93.30,16.092,48.645,26.94,74.32,136.10,96.53,138.85,40.620,66.42,48.41,39.46,112.74,31.730
2021-05-05,67.28,96.16,94.06,16.412,50.880,27.26,75.58,140.00,96.70,141.20,41.395,66.08,49.09,39.56,115.84,32.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-13,58.08,80.16,89.96,16.026,54.830,22.73,77.98,151.50,117.24,196.55,36.725,57.74,60.02,41.96,120.50,40.050
2022-01-14,56.26,82.34,90.72,15.962,54.130,22.32,77.78,145.86,117.28,191.15,37.355,58.96,59.58,41.96,120.48,39.245
2022-01-17,55.50,80.00,90.14,15.946,54.200,22.61,79.36,146.00,116.80,193.85,37.450,60.66,59.56,43.15,122.52,39.570


In [10]:
min_weight, max_weight = 0.01, 0.20
print("--------------------------------------")
print(f"Weight range: [{min_weight}, {max_weight}]")
print()
mu = expected_returns.mean_historical_return(close_prices_12m)
S = risk_models.CovarianceShrinkage(close_prices_12m).ledoit_wolf()

ef = EfficientFrontier(expected_returns=mu, cov_matrix=S, weight_bounds=(min_weight, max_weight))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
print(pd.Series(cleaned_weights).sort_values(ascending=False))

--------------------------------------
Weight range: [0.01, 0.2]

Expected annual return: 23.2%
Annual volatility: 13.9%
Sharpe Ratio: 1.52
AIRG.DE      0.20
MRCG.DE      0.20
SHLG.DE      0.20
QIA.DE       0.20
DPWGn.DE     0.09
HFGG.DE      0.01
HNKG_p.DE    0.01
BEIG.DE      0.01
DTEGn.DE     0.01
ENR1n.DE     0.01
BNRGn.DE     0.01
SIEGn.DE     0.01
FREG.DE      0.01
FMEG.DE      0.01
SAPG.DE      0.01
IFXGn.DE     0.01
dtype: float64


In [11]:
print(pd.Series(cleaned_weights).sort_values(ascending=False)[:8])

AIRG.DE      0.20
MRCG.DE      0.20
SHLG.DE      0.20
QIA.DE       0.20
DPWGn.DE     0.09
HFGG.DE      0.01
HNKG_p.DE    0.01
BEIG.DE      0.01
dtype: float64


In [12]:
curr_prices = ek.get_data(final_companies1, "CF_LAST")[0]
curr_prices = curr_prices.set_index("Instrument")['CF_LAST']

In [13]:
curr_prices

Instrument
HFGG.DE       50.82
HNKG_p.DE     74.32
BEIG.DE       87.02
DTEGn.DE     17.554
DPWGn.DE      53.19
ENR1n.DE       19.3
BNRGn.DE      77.58
SIEGn.DE     144.68
AIRG.DE      118.28
MRCG.DE      185.25
FREG.DE      37.295
FMEG.DE       59.06
SHLG.DE        57.6
QIA.DE        43.81
SAPG.DE       109.3
IFXGn.DE     34.165
Name: CF_LAST, dtype: Float64

In [14]:
da = DiscreteAllocation(cleaned_weights, curr_prices, total_portfolio_value=60000)
allocation, leftover = da.lp_portfolio()
print("Left Over: {}".format(leftover))

Left Over: 6.025000000001455


In [15]:
df = pd.DataFrame()
df['Number of shares'] = pd.Series(allocation)
df['Weights'] = pd.Series(cleaned_weights).sort_values(ascending=False)
df['Current Price'] = pd.Series(curr_prices)
df

,Number of shares,Weights,Current Price
HFGG.DE,12,0.01,50.82
HNKG_p.DE,8,0.01,74.32
BEIG.DE,7,0.01,87.02
DTEGn.DE,35,0.01,17.554
DPWGn.DE,101,0.09,53.19
ENR1n.DE,31,0.01,19.3
BNRGn.DE,8,0.01,77.58
SIEGn.DE,4,0.01,144.68
AIRG.DE,101,0.20,118.28
MRCG.DE,65,0.20,185.25


In [16]:
(df['Number of shares'] * df['Current Price']).sum()
# df.to_csv("optimisation1.csv")

59993.975000000006